[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sci-ndp/ndp-ep-py/blob/main/docs/source/tutorials/bulk_resource_management.ipynb)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/sci-ndp/ndp-ep-py/main?filepath=docs%2Fsource%2Ftutorials%2Fbulk_resource_management.ipynb)

# Bulk Resource Management Tutorial

Welcome to the advanced tutorial on bulk resource management with the NDP EP Python client!

## 🎯 What You'll Learn

This tutorial demonstrates real-world scenarios for managing large numbers of resources:

- **🏢 Organization Management**: Create and manage organizations for bulk operations
- **📊 Bulk Service Registration**: Register 100 services efficiently
- **📁 Bulk Dataset Registration**: Register 100 datasets with proper organization
- **📈 Progress Monitoring**: Track operations and handle errors gracefully
- **✅ Verification**: Confirm all resources were created successfully
- **🧹 Complete Cleanup**: Remove all created resources to leave system clean

## 🔧 Use Cases

This tutorial is perfect for:
- **Data Migration**: Moving large datasets between systems
- **System Setup**: Initial configuration with multiple resources
- **Testing Environments**: Creating test data for development
- **Batch Operations**: Learning patterns for bulk data management

## ⚠️ Prerequisites

- Valid NDP EP API credentials (token)
- Write permissions on the target API instance
- Basic understanding of the NDP EP API (complete the getting started tutorial first)

## 🛡️ Safety Note

This tutorial creates and then deletes 200+ resources. It's designed to be safe and self-contained, but always test on non-production systems first.

In [1]:
# Install required libraries
!pip install ndp-ep

# Import required modules
import time
import getpass
from typing import List, Dict, Any, Optional
from ndp_ep import APIClient

print("✅ Libraries installed and imported successfully!")
print("📚 Ready to start bulk resource management tutorial")

✅ Libraries installed and imported successfully!
📚 Ready to start bulk resource management tutorial


## 1. 🔐 Secure Authentication Setup

First, let's configure the client with your credentials. Your token will be hidden during input for security.

In [2]:
# Interactive API configuration
print("🔧 API Configuration")
print("=" * 30)

# Get API base URL
api_url = input("Enter API base URL [http://localhost:8003]: ").strip()
if not api_url:
    api_url = "http://localhost:8003"

print(f"📡 API URL: {api_url}")

# Get API token securely
print("\n🔐 Authentication")
print("Please enter your API token (it will be hidden):")
api_token = getpass.getpass("API Token: ")

if not api_token.strip():
    raise ValueError("❌ API token is required for this tutorial")

print("✅ Credentials configured securely")

🔧 API Configuration
📡 API URL: http://localhost:8003

🔐 Authentication
Please enter your API token (it will be hidden):
✅ Credentials configured securely


In [3]:
# Initialize and test the API client
print("🚀 Initializing API Client...")

try:
    client = APIClient(base_url=api_url, token=api_token)
    
    # Test connection
    system_status = client.get_system_status()
    print("✅ API client initialized successfully")
    print(f"🌐 Connected to: {api_url}")
    print("🔑 Authentication verified")
    
except Exception as e:
    print(f"❌ Failed to initialize client: {e}")
    print("💡 Please check your credentials and API URL")
    raise

🚀 Initializing API Client...
✅ API client initialized successfully
🌐 Connected to: http://localhost:8003
🔑 Authentication verified


## 2. 📋 Tutorial Configuration

Let's set up the parameters for our bulk operations.

In [4]:
# Configuration for bulk operations
SERVICES_COUNT = 100
DATASETS_COUNT = 100
SERVICES_ORG = "services"  # Default org for services
DATASETS_ORG = "test_services"  # Custom org for datasets
PREFIX = "test_"  # Prefix for all resource names

print("📊 Bulk Operation Configuration")
print("=" * 35)
print(f"Services to create: {SERVICES_COUNT}")
print(f"Datasets to create: {DATASETS_COUNT}")
print(f"Services organization: {SERVICES_ORG}")
print(f"Datasets organization: {DATASETS_ORG}")
print(f"Name prefix: {PREFIX}")
print(f"Total resources: {SERVICES_COUNT + DATASETS_COUNT}")

# Storage for tracking created resources
created_services = []
created_datasets = []
operation_log = []

📊 Bulk Operation Configuration
Services to create: 100
Datasets to create: 100
Services organization: services
Datasets organization: test_services
Name prefix: test_
Total resources: 200


## 3. 🏗️ Utility Functions

Let's create helper functions for our bulk operations.

In [5]:
def log_operation(operation: str, resource_type: str, 
                  resource_name: str, success: bool, 
                  details: str = "") -> None:
    """
    Log an operation for tracking and debugging.
    
    Args:
        operation: Type of operation (create, delete, verify)
        resource_type: Type of resource (service, dataset, organization)
        resource_name: Name of the resource
        success: Whether the operation was successful
        details: Additional details or error messages
    """
    timestamp = time.strftime("%H:%M:%S")
    status = "✅" if success else "❌"
    
    log_entry = {
        "timestamp": timestamp,
        "operation": operation,
        "resource_type": resource_type,
        "resource_name": resource_name,
        "success": success,
        "details": details
    }
    
    operation_log.append(log_entry)
    print(f"{status} [{timestamp}] {operation.title()} {resource_type}: {resource_name}")
    
    if details and not success:
        print(f"   └─ Error: {details}")


def generate_service_data(index: int) -> Dict[str, Any]:
    """
    Generate service data for registration.
    
    Args:
        index: Service index number
        
    Returns:
        Service data dictionary
    """
    service_num = str(index).zfill(3)  # Pad with zeros: 001, 002, etc.
    
    return {
        "service_name": f"{PREFIX}service_{service_num}",
        "service_title": f"{PREFIX.title()}Service {service_num}",
        "owner_org": SERVICES_ORG,
        "service_url": f"https://api.example.com/service_{service_num}",
        "service_type": "REST API",
        "notes": f"Bulk tutorial test service number {service_num}",
        "health_check_url": f"https://api.example.com/service_{service_num}/health",
        "documentation_url": f"https://docs.example.com/service_{service_num}"
    }


def generate_dataset_data(index: int) -> Dict[str, Any]:
    """
    Generate dataset data for registration.
    
    Args:
        index: Dataset index number
        
    Returns:
        Dataset data dictionary
    """
    dataset_num = str(index).zfill(3)  # Pad with zeros: 001, 002, etc.
    
    return {
        "name": f"{PREFIX}dataset_{dataset_num}",
        "title": f"{PREFIX.title()}Dataset {dataset_num}",
        "owner_org": DATASETS_ORG,
        "notes": f"Bulk tutorial test dataset number {dataset_num}",
        "tags": [{"name": "test"}, {"name": "bulk"}, {"name": "tutorial"}],
        "license_id": "cc-by",
        "private": False
    }


def print_progress(current: int, total: int, 
                   operation: str = "Processing") -> None:
    """
    Print progress information.
    
    Args:
        current: Current item number
        total: Total items to process
        operation: Description of the operation
    """
    percentage = (current / total) * 100
    bar_length = 30
    filled_length = int(bar_length * current // total)
    
    bar = "█" * filled_length + "-" * (bar_length - filled_length)
    print(f"\r{operation}: |{bar}| {current}/{total} ({percentage:.1f}%)", end="")
    
    if current == total:
        print()  # New line when complete


print("🔧 Utility functions defined successfully")
print("📝 Ready for bulk operations")

🔧 Utility functions defined successfully
📝 Ready for bulk operations


## 4. 🏢 Organization Setup

Before creating datasets, we need to ensure the organization exists.

In [6]:
# Check if datasets organization exists, create if needed
print(f"🏢 Setting up organization: {DATASETS_ORG}")
print("=" * 40)

try:
    # Check existing organizations
    existing_orgs = client.list_organizations(server="local")
    
    if DATASETS_ORG in existing_orgs:
        print(f"✅ Organization '{DATASETS_ORG}' already exists")
        log_operation("verify", "organization", DATASETS_ORG, True, 
                     "Already exists")
    else:
        # Create the organization
        print(f"🔨 Creating organization '{DATASETS_ORG}'...")
        
        org_data = {
            "name": DATASETS_ORG,
            "title": "Test Services Organization",
            "description": "Organization for bulk tutorial test datasets"
        }
        
        result = client.register_organization(org_data, server="local")
        print(f"✅ Organization created with ID: {result.get('id', 'unknown')}")
        log_operation("create", "organization", DATASETS_ORG, True, 
                     f"ID: {result.get('id', 'unknown')}")

except Exception as e:
    print(f"❌ Failed to setup organization: {e}")
    log_operation("create", "organization", DATASETS_ORG, False, str(e))
    raise

print("\n🎯 Organization setup complete!")

🏢 Setting up organization: test_services
🔨 Creating organization 'test_services'...
✅ Organization created with ID: 504ac9b3-d382-4b37-9fb9-771252316780
✅ [15:14:35] Create organization: test_services

🎯 Organization setup complete!


---

**🎉 Base Tutorial Structure Complete!**

The foundation is ready. Next commits will add:
- Bulk service registration
- Bulk dataset registration  
- Verification and monitoring
- Complete cleanup process